In [28]:
import numpy as np
import pandas as pd
from chefboost import Chefboost as chef
from sklearn.compose import make_column_transformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
 

training_df = pd.read_csv('NaiveBayesian_training.csv',
                          usecols=[3,4,5,6])
retest_df = pd.read_csv('NaiveBayesian_test.csv',
                          usecols=[3,4,5,6])
test_df = pd.read_csv('NaiveBayesian_test.csv',
                          usecols=[3,4,5])

# (1) “Is Home/Away?”, a 2-value attribute (“Home”, “Away”), 
# (2) “Is Opponent in AP Top 25 at Preseason?”, a 2-value attribute (“In”, “Out”), 
# (3) “Media”, a 5-value attribute (“1-NBC”, “2- ESPN”, “3-FOX”, “4-ABC”, “5-CBS”). The label “Win/Lose” is binary (“Win”, “Lose”). 

training_df.head()

,Is_Home_or_Away,Is_Opponent_in_AP25 _Preseason,Media,Label
0,Home,Out,1-NBC,1
1,Away,Out,4-ABC,1
2,Home,In,1-NBC,1
3,Home,Out,1-NBC,1
4,Away,In,4-ABC,0


In [29]:
# Rename training dataframe label column and the data values to Decision and change yes=1 and lose=0

training_df=training_df.rename(columns={"Label":"Decision"})
# training_df['Decision'] = training_df['Decision'].map({'Win': 1, 'Lose': 0})
training_df.head()

,Is_Home_or_Away,Is_Opponent_in_AP25 _Preseason,Media,Decision
0,Home,Out,1-NBC,1
1,Away,Out,4-ABC,1
2,Home,In,1-NBC,1
3,Home,Out,1-NBC,1
4,Away,In,4-ABC,0


In [30]:
# enc = OrdinalEncoder()
# X = [['Male', 1], ['Female', 3], ['Female', 2]]
# enc.fit(X)
# OrdinalEncoder()
# enc.categories_
# [array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]
# enc.transform([['Female', 3], ['Male', 1]])
# array([[0., 2.],
#        [1., 0.]])

ohe= OneHotEncoder(sparse=False)
ohe.fit_transform(training_df[['Is_Home_or_Away']])


array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.]])

In [31]:
ohe.fit_transform(training_df[['Is_Opponent_in_AP25 _Preseason']])


array([[0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [32]:
ohe.fit_transform(training_df[['Media']])


array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

In [33]:
ohe.fit_transform(training_df[['Decision']])

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [34]:
x= training_df.drop(columns= 'Decision')
y= training_df.Decision

In [35]:
ohe.fit_transform(x)

array([[0., 1., 0., 1., 1., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0., 1., 0.],
       [0., 1., 1., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 1., 1., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0., 1., 0.],
       [1., 0., 0., 1., 0., 0., 0., 1., 0.],
       [0., 1., 0., 1., 1., 0., 0., 0., 0.],
       [1., 0., 0., 1., 1., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0., 1., 0.],
       [0., 1., 0., 1., 1., 0., 0., 0., 0.],
       [0., 1., 0., 1., 1., 0., 0., 0., 0.],
       [0., 1., 0., 1., 1., 0., 0., 0., 0.],
       [0., 1., 0., 1., 0., 1., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0., 1., 0.],
       [0., 1., 1., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 1., 1., 0., 0., 0., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0., 1.],
       [0., 1., 0., 1., 1., 0., 0., 0., 0.],
       [0.

In [36]:
column_trans = make_column_transformer(
    (OneHotEncoder(), ['Is_Home_or_Away','Is_Opponent_in_AP25 _Preseason','Media']),
    remainder = 'passthrough')
multi_nom= MultinomialNB()

In [37]:
pipe = make_pipeline(column_trans, multi_nom)
x=column_trans.fit_transform(x)
multi_nom.fit(x, y)
# logreg = LogisticRegression(solver='lbfgs')
# pipe = make_pipeline(column_trans, logreg )


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [38]:
# from sklearn import preprocessing
# #creating labelEncoder
# le = preprocessing.LabelEncoder()
# home_away_encoded=le.fit_transform()
# print (home_away_encoded)

In [39]:
multi_nom.fit(x, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [40]:
print(multi_nom.predict(x))
# win=1, lose=0

[1 0 1 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 1 1 1 0]


In [41]:
# ohe= OneHotEncoder(sparse=False)
# ohe.fit_transform(retest_df[['Is_Home_or_Away']])
# ohe.fit_transform(retest_df[['Is_Opponent_in_AP25 _Preseason']])
# ohe.fit_transform(retest_df[['Media']])
# ohe.fit_transform(retest_df[['Decision']])


# x= training_df.drop(columns= 'Label')
# y= training_df.Label


In [42]:
# x_test=test_df
# y_test=retest_df.Decision
# ohe.fit_transform(x_test)


# print(multi_nom.predict(x_test))


In [43]:
# # Rename test dataframe label column and the data values to Decision and change yes=1 and lose=0
retest_df=retest_df.rename(columns={"Label":"Decision"})
retest_df.head()

,Is_Home_or _Away,Is_Opponent_in_AP25 _Preseason,Media,Decision
0,Home,Out,1-NBC,1
1,Home,In,1-NBC,0
2,Away,Out,2-ESPN,1
3,Away,Out,3-FOX,1
4,Home,Out,1-NBC,1


In [44]:
test_df.head()

,Is_Home_or _Away,Is_Opponent_in_AP25 _Preseason,Media
0,Home,Out,1-NBC
1,Home,In,1-NBC
2,Away,Out,2-ESPN
3,Away,Out,3-FOX
4,Home,Out,1-NBC


In [47]:
# print('Actual Values')
# print(retest_df['Decision'])

# print('Predicted Values')
# for index, instance in retest_df.iloc[:,: ].iterrows():
#     prediction = chef.predict(model, instance)
#     print(prediction)

In [55]:
from sklearn.metrics import f1_score
y_true = multi_nom.predict(x)
y_pred = y
print(f1_score(y_true, y_pred, average='macro'))
print(f1_score(y_true, y_pred, average='micro'))
print(f1_score(y_true, y_pred, average='weighted'))
print(f1_score(y_true, y_pred, average=None))


0.6444444444444445
0.6666666666666666
0.674074074074074
[0.55555556 0.73333333]


In [56]:
config = {'algorithm': 'C4.5'}
model = chef.fit(retest_df, config)

Regression  tree is going to be built...
MAE:  0.125
RMSE:  0.25
Mean:  0.75
MAE / Mean:  16.666666666666668 %
RMSE / Mean:  33.333333333333336 %
finished in  0.13023042678833008  seconds


In [57]:
# ID3 algorithm

config = {'algorithm': 'ID3'}
model = chef.fit(retest_df, config)

Regression  tree is going to be built...
MAE:  0.125
RMSE:  0.25
Mean:  0.75
MAE / Mean:  16.666666666666668 %
RMSE / Mean:  33.333333333333336 %
finished in  0.11471819877624512  seconds
